# 서울시 미용실 데이터 - NAVER Map(v4) 데이터 전처리(2)
## 작성자(최종수정일) : 서동원(2020.04.20)
- 검색 데이터 필요한 정보 추려내기
- (1) 중복 제거하기
- (2) 카테고리가 미용실 관련 데이터만 남기기
- (3) 주소가 서울인 데이터만 남기기

In [21]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import re
import requests
import time
pd.set_option('display.max_columns',50) # DataFrame truncation 없이 보기
pd.set_option('display.max_rows',100)
from tqdm import tqdm
import math

## 네이버 검색 데이터

In [24]:
# 파일 읽어오기
naver_search_raw = pd.read_csv('./data/naver_map_seoul_salon_output_data_2020-04-20_modified.csv', index_col=0)
pre_count = len(naver_search_raw)

In [25]:
# 중복제거
naver_search_raw.drop_duplicates(inplace=True)
print('중복 제거 전 개수 :', pre_count)
print('중복 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

중복 제거 전 개수 : 26902
중복 제거 후 개수 : 24215


In [26]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24215 entries, 0 to 105
Data columns (total 10 columns):
업소ID       24215 non-null object
업소명        24213 non-null object
소재지전화번호    16946 non-null object
소재지도로명     24215 non-null object
카테고리       24215 non-null object
검색인덱스      24215 non-null int64
검색어        24215 non-null object
검색건수       24215 non-null int64
수집일        24215 non-null object
검색주소타입     106 non-null float64
dtypes: float64(1), int64(2), object(7)
memory usage: 2.0+ MB


In [27]:
# '미용' 단어가 들어간 카테고리만 뽑기
list(filter(lambda x: '미용' in x, naver_search_raw['카테고리'].unique()))

['생활,편의 > 미용실',
 '미용 > 미용실',
 '미용 > 머리염색',
 '미용 > 네일아트,네일샵',
 '미용 > 메이크업',
 '생활,편의 > 미용',
 '미용 > 피부,체형관리',
 '미용 > 마사지,지압',
 '미용 > 속눈썹증모,연장',
 '생활,편의 > 남성전문미용실',
 '미용 > 두피,탈모관리',
 '미용 > 경락마사지',
 '쇼핑,유통 > 미용기기,재료',
 '협회,단체 > 화장품,미용',
 '직업,기술학원 > 미용',
 '미용실 > 어린이미용실',
 '반려동물 > 반려동물미용',
 '미용 > 다이어트,비만',
 '직업,기술학원 > 애견미용']

### 미용이 들어간 카테고리 중 실제로 필요한 카테고리는 다음과 같다.
    - (1) '생활,편의 > 미용실'
    - (2) '미용 > 미용실'
    - (3) '미용 > 머리염색'
    - (4) '생활,편의 > 미용'
    - (5) '생활,편의 > 남성전문미용실'
    - (6) '미용실 > 어린이미용실'

In [28]:
# 필요한 카케고리 리스트로 저장
category_list = ['생활,편의 > 미용실'
                , '미용 > 미용실'
                , '미용 > 머리염색'
                , '생활,편의 > 미용'
                , '생활,편의 > 남성전문미용실'
                , '미용실 > 어린이미용실']
category_list

['생활,편의 > 미용실',
 '미용 > 미용실',
 '미용 > 머리염색',
 '생활,편의 > 미용',
 '생활,편의 > 남성전문미용실',
 '미용실 > 어린이미용실']

In [29]:
# 카테고리로 제거하기
naver_search_raw.drop(
      index=naver_search_raw[~naver_search_raw['카테고리'].isin(category_list)].index
    , inplace=True
)
print('카테고리로 제거 전 개수 :', pre_count)
print('카테고리로 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

카테고리로 제거 전 개수 : 24215
카테고리로 제거 후 개수 : 21783


In [30]:
# 카테고리로 확인
# 남긴 카테고리 이외에는 검색되지 않는다.
naver_search_raw['카테고리'].unique()

array(['생활,편의 > 미용실', '미용 > 미용실', '미용 > 머리염색', '생활,편의 > 미용',
       '생활,편의 > 남성전문미용실', '미용실 > 어린이미용실'], dtype=object)

In [31]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21783 entries, 0 to 105
Data columns (total 10 columns):
업소ID       21783 non-null object
업소명        21781 non-null object
소재지전화번호    15252 non-null object
소재지도로명     21783 non-null object
카테고리       21783 non-null object
검색인덱스      21783 non-null int64
검색어        21783 non-null object
검색건수       21783 non-null int64
수집일        21783 non-null object
검색주소타입     44 non-null float64
dtypes: float64(1), int64(2), object(7)
memory usage: 1.8+ MB


In [32]:
# 네이버 지도에 검색할떄 사용한 업소명인 공공데이터 자체에 중복데이터가 있을 수 있다.
# '업소ID', '업소명', '소재지전화번호', '소재지도로명', '카테고리'
# 5가지 컬럼으로 팬시색인하여 중복값을 다시 한번 제거한다.
naver_search_raw.drop(
      index = naver_search_raw[naver_search_raw[['업소ID', '업소명', '소재지전화번호', '소재지도로명', '카테고리']].duplicated()].index
    , inplace=True
)
print('팬시색인으로 중복 제거 전 개수 :', pre_count)
print('팬시색인으로 중복 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

팬시색인으로 중복 제거 전 개수 : 21783
팬시색인으로 중복 제거 후 개수 : 14130


In [33]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14130 entries, 0 to 104
Data columns (total 10 columns):
업소ID       14130 non-null object
업소명        14129 non-null object
소재지전화번호    10069 non-null object
소재지도로명     14130 non-null object
카테고리       14130 non-null object
검색인덱스      14130 non-null int64
검색어        14130 non-null object
검색건수       14130 non-null int64
수집일        14130 non-null object
검색주소타입     13 non-null float64
dtypes: float64(1), int64(2), object(7)
memory usage: 1.2+ MB


In [34]:
# 업소명에 NaN이 들어있어 네이버에 업소ID로 검색해본 결과
# 네이버 플레이스에도 이름이 nan으로 처리되어있다. 네이버 자체의 실수인듯
# 확인 URL : https://store.naver.com/hairshops/detail?id=472920069
naver_search_raw[naver_search_raw['업소명'].isnull()]

,업소ID,업소명,소재지전화번호,소재지도로명,카테고리,검색인덱스,검색어,검색건수,수집일,검색주소타입
15848,s472920069,NaN,NaN,서울특별시 서대문구 연희로11길 30,"생활,편의 > 미용실",8640,헤어아트,32,2020-04-19,NaN


In [35]:
# 주소에 '서울' 문자열이 포함되지 않은 데이터 제거
naver_search_raw = naver_search_raw[naver_search_raw['소재지도로명'].apply(lambda x: '서울' in x.split()[0])]
print('주소가 서울이 아닌 정보 제거 전 개수 :', pre_count)
print('주소가 서울이 아닌 정보 제거 후 개수 :', len(naver_search_raw))
pre_count = len(naver_search_raw)

주소가 서울이 아닌 정보 제거 전 개수 : 14130
주소가 서울이 아닌 정보 제거 후 개수 : 13399


In [36]:
naver_search_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13399 entries, 0 to 104
Data columns (total 10 columns):
업소ID       13399 non-null object
업소명        13398 non-null object
소재지전화번호    9515 non-null object
소재지도로명     13399 non-null object
카테고리       13399 non-null object
검색인덱스      13399 non-null int64
검색어        13399 non-null object
검색건수       13399 non-null int64
수집일        13399 non-null object
검색주소타입     13 non-null float64
dtypes: float64(1), int64(2), object(7)
memory usage: 1.1+ MB


In [37]:
# 저장을 위해 필요한 컬럼만 컬럼명을 바꿔서 팬시색인한다.
naver_search_for_save = naver_search_raw[['업소ID', '업소명', '소재지전화번호', '소재지도로명', '카테고리']].rename(columns={
      '업소ID' : 'store_id'
    , '업소명' : 'store_name'
    , '소재지전화번호' : 'tel'
    , '소재지도로명' : 'addrRoad'
    , '카테고리' : 'category'
})

naver_search_for_save

,store_id,store_name,tel,addrRoad,category
0,s18570093,조희미용실,02-555-0157,서울특별시 강남구 남부순환로 2917 1층 111-가,"생활,편의 > 미용실"
1,s10983326,정정원 헤어 룩,02-545-3809,서울특별시 강남구 선릉로116길 24,"생활,편의 > 미용실"
15,s18431392,헤어인러브,02-546-0366,서울특별시 강남구 선릉로148길 39 2층,"생활,편의 > 미용실"
22,s34786519,앤즈헤어,02-553-9799,서울특별시 강남구 언주로 201 sk리더스뷰 2층 앤즈헤어,"생활,편의 > 미용실"
24,s697084146,아나스타일,02-501-0457,서울특별시 강남구 언주로97길 25,"생활,편의 > 미용실"
...,...,...,...,...,...
31,s20925215,전혜정머리나라,02-706-5712,서울특별시 마포구 숭문길 42,"생활,편의 > 미용실"
32,s20890996,주연미용실,02-718-0827,서울특별시 마포구 숭문길 58,"생활,편의 > 미용실"
36,s1513007561,홍대샵,02-334-0740,서울특별시 마포구 와우산로23길 9 203호 홍대샵,"생활,편의 > 미용실"
101,s19408439,한빛미용실,NaN,서울특별시 중구 퇴계로53길 6-11,"생활,편의 > 미용실"


In [38]:
naver_search_for_save['collect_day'] = '2020-04-19'
naver_search_for_save

,store_id,store_name,tel,addrRoad,category,collect_day
0,s18570093,조희미용실,02-555-0157,서울특별시 강남구 남부순환로 2917 1층 111-가,"생활,편의 > 미용실",2020-04-19
1,s10983326,정정원 헤어 룩,02-545-3809,서울특별시 강남구 선릉로116길 24,"생활,편의 > 미용실",2020-04-19
15,s18431392,헤어인러브,02-546-0366,서울특별시 강남구 선릉로148길 39 2층,"생활,편의 > 미용실",2020-04-19
22,s34786519,앤즈헤어,02-553-9799,서울특별시 강남구 언주로 201 sk리더스뷰 2층 앤즈헤어,"생활,편의 > 미용실",2020-04-19
24,s697084146,아나스타일,02-501-0457,서울특별시 강남구 언주로97길 25,"생활,편의 > 미용실",2020-04-19
...,...,...,...,...,...,...
31,s20925215,전혜정머리나라,02-706-5712,서울특별시 마포구 숭문길 42,"생활,편의 > 미용실",2020-04-19
32,s20890996,주연미용실,02-718-0827,서울특별시 마포구 숭문길 58,"생활,편의 > 미용실",2020-04-19
36,s1513007561,홍대샵,02-334-0740,서울특별시 마포구 와우산로23길 9 203호 홍대샵,"생활,편의 > 미용실",2020-04-19
101,s19408439,한빛미용실,NaN,서울특별시 중구 퇴계로53길 6-11,"생활,편의 > 미용실",2020-04-19


In [46]:
index_ = naver_search_for_save[naver_search_for_save['store_name'].isnull()].index
index_

Int64Index([15848], dtype='int64')

In [47]:
naver_search_for_save.drop(index=index_, inplace=True)

In [52]:
naver_search_for_save.index = range(len(naver_search_for_save))
naver_search_for_save.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13398 entries, 0 to 13397
Data columns (total 6 columns):
store_id       13398 non-null object
store_name     13398 non-null object
tel            9515 non-null object
addrRoad       13398 non-null object
category       13398 non-null object
collect_day    13398 non-null object
dtypes: object(6)
memory usage: 628.2+ KB


In [53]:
naver_search_for_save.tail(2)

,store_id,store_name,tel,addrRoad,category,collect_day
13396,s19408439,한빛미용실,NaN,서울특별시 중구 퇴계로53길 6-11,"생활,편의 > 미용실",2020-04-19
13397,s20834916,준 미용실,02-753-8045,서울특별시 중구 퇴계로 27 804호,"생활,편의 > 미용실",2020-04-19


In [54]:
# DF를 csv파일로 저장
naver_search_for_save.to_csv(
      './data/naver_map_seoul_salon_output_data_2020-04-19_final.csv'
    , sep=','
    , encoding='utf-8'
)

In [55]:
# 데이터 확인
pd.read_csv('./data/naver_map_seoul_salon_output_data_2020-04-19_final.csv', index_col=0)


,store_id,store_name,tel,addrRoad,category,collect_day
0,s18570093,조희미용실,02-555-0157,서울특별시 강남구 남부순환로 2917 1층 111-가,"생활,편의 > 미용실",2020-04-19
1,s10983326,정정원 헤어 룩,02-545-3809,서울특별시 강남구 선릉로116길 24,"생활,편의 > 미용실",2020-04-19
2,s18431392,헤어인러브,02-546-0366,서울특별시 강남구 선릉로148길 39 2층,"생활,편의 > 미용실",2020-04-19
3,s34786519,앤즈헤어,02-553-9799,서울특별시 강남구 언주로 201 sk리더스뷰 2층 앤즈헤어,"생활,편의 > 미용실",2020-04-19
4,s697084146,아나스타일,02-501-0457,서울특별시 강남구 언주로97길 25,"생활,편의 > 미용실",2020-04-19
...,...,...,...,...,...,...
13393,s20925215,전혜정머리나라,02-706-5712,서울특별시 마포구 숭문길 42,"생활,편의 > 미용실",2020-04-19
13394,s20890996,주연미용실,02-718-0827,서울특별시 마포구 숭문길 58,"생활,편의 > 미용실",2020-04-19
13395,s1513007561,홍대샵,02-334-0740,서울특별시 마포구 와우산로23길 9 203호 홍대샵,"생활,편의 > 미용실",2020-04-19
13396,s19408439,한빛미용실,NaN,서울특별시 중구 퇴계로53길 6-11,"생활,편의 > 미용실",2020-04-19
